<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/main/Plasma_vortex_in_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define simulation parameters
Lx = 20         # Simulation domain length (m)
Ly = 20
Nx = 128        # Number of grid points
Ny = 128
dx = Lx/Nx      # Grid spacing
dy = Ly/Ny
dt = 1e-11      # Time step
nsteps = 1000   # Number of time steps
q = -1.602e-19  # Electron charge (C)
m = 9.109e-31   # Electron mass (kg)
epsilon0 = 8.854e-12  # Vacuum permittivity (F/m)
B0 = 0.1        # Magnetic field strength (T)

# Define initial conditions
x = np.linspace(0, Lx, Nx, endpoint=False)
y = np.linspace(0, Ly, Ny, endpoint=False)
X, Y = np.meshgrid(x, y)
vx = np.zeros_like(X)
vy = np.zeros_like(Y)
n0 = 1e18       # Plasma density (m^-3)
n = n0*np.exp(-(X-Lx/2)**2/(2*5**2) - (Y-Ly/2)**2/(2*5**2))
q_n = q*n
ex = np.zeros_like(n)
ey = np.zeros_like(n)
bx = np.zeros_like(n)
by = np.zeros_like(n)
omega_c = q*B0/m
vx += omega_c*(Y-Ly/2)
vy -= omega_c*(X-Lx/2)

# Define function to interpolate electric and magnetic fields to particle positions
def interpolate_fields(x, y, ex, ey, bx, by):
    ix = np.floor(x/dx).astype(int)
    iy = np.floor(y/dy).astype(int)
    fx = (x/dx - ix).reshape(-1, 1)
    fy = (y/dy - iy).reshape(-1, 1)
    ex_interp = (1-fx)*(1-fy)*ex[ix, iy] + fx*(1-fy)*ex[ix+1, iy] \
              + (1-fx)*fy*ex[ix, iy+1] + fx*fy*ex[ix+1, iy+1]
    ey_interp = (1-fx)*(1-fy)*ey[ix, iy] + fx*(1-fy)*ey[ix+1, iy] \
              + (1-fx)*fy*ey[ix, iy+1] + fx*fy*ey[ix+1, iy+1]
    bx_interp = (1-fx)*(1-fy)*bx[ix, iy] + fx*(1-fy)*bx[ix+1, iy] \
              + (1-fx)*fy*bx[ix, iy+1] + fx*fy*bx[ix+1, iy+1]
    by_interp = (1-fx)*(1-fy)*by[ix, iy] + fx*(1-fy)*by[ix+1, iy] \
              + (1-fx)*fy*by[ix, iy+1] + fx*fy*by[ix+1, iy+1]
    return ex_interp, ey_interp, bx_interp, by_interp

# Define function to update particle velocities and positions
def update_particles(x, y, vx, vy, q_n, ex, ey, bx, by, dt):
# Interpolate fields to particle positions
ex_interp, ey_interp, bx_interp, by_interp = interpolate_fields(x, y, ex, ey, bx, by)
# Update velocities using the Lorentz force
vx += q_n/m*(ex_interp + vyomega_c - vyby_interp)
vy += q_n/m*(ey_interp - vxomega_c + vxbx_interp)
# Update positions using the updated velocities
x += vxdt
y += vydt
# Apply periodic boundary conditions
x[x < 0] += Lx
x[x >= Lx] -= Lx
y[y < 0] += Ly
y[y >= Ly] -= Ly
return x, y, vx, vy

Define function to solve Poisson's equation for the electric potential
def solve_poisson(n, epsilon0):
rho = q*(n - n0)
rho_hat = np.fft.fft2(rho)
kx = 2np.pinp.fft.fftfreq(Nx, dx)
ky = 2np.pinp.fft.fftfreq(Ny, dy)
kx, ky = np.meshgrid(kx, ky, indexing='ij')
k2 = kx2 + ky2
phi_hat = -rho_hat/(k2*epsilon0)
phi_hat[0, 0] = 0
phi = np.fft.ifft2(phi_hat).real
ex = -np.gradient(phi, dx, axis=1)
ey = -np.gradient(phi, dy, axis=0)
return ex, ey

Initialize particle positions and velocities
n_particles = 10000
x_particles = np.random.uniform(0, Lx, n_particles)
y_particles = np.random.uniform(0, Ly, n_particles)
vx_particles = np.zeros(n_particles)
vy_particles = np.zeros(n_particles)

Main loop
for i in range(nsteps):
# Solve Poisson's equation for the electric potential
ex, ey = solve_poisson(n, epsilon0)
# Update particle velocities and positions
x_particles, y_particles, vx_particles, vy_particles = update_particles(
x_particles, y_particles, vx_particles, vy_particles, q_n, ex, ey, bx, by, dt)
# Update magnetic field due to electron vortices
bx = -np.gradient(vy_particles, dx, axis=1)
by = np.gradient(vx_particles, dy, axis=0)
# Plot particle positions
if i % 10 == 0:
plt.clf()
plt.pcolormesh(X, Y, n, cmap='Blues')
plt.quiver(X[::4, ::4], Y[::4, ::4], ex[::4, ::4], ey[::4, ::4], scale=50)
plt.quiver(X[::4, ::4], Y[::4, ::4], bx[::4, ::4], by[::4, ::4], color='red', scale=50)
plt.plot(x_particles, y_particles, '.', markersize=0.5, color='white')
plt.xlim(0, Lx)
plt.ylim(0, Ly)
plt.title('Time step %d' % i)
plt.pause(0.001)

plt.show()
